In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os

from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image

from keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from keras.models import Sequential, Model
from keras.applications import ResNet50

from keras import backend as K

Using TensorFlow backend.


In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

                                Parameters

In [3]:
PATH = 'data/dogs_vs_cats/'
imsize = (224, 224)
batch_size = 64
train_data_dir = os.path.join(PATH, 'train')
valid_data_dir = os.path.join(PATH, 'valid')

In [4]:
epochs = 10
batch_size = 8
workers = 4

In [5]:
# Generate data with normalization data augmentation (shear, zoom, horizontal flip)
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
valid_datagen = ImageDataGenerator(rescale=1./255)
# Generate images from train / valid directories
train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                    target_size=imsize,
                                                    batch_size=batch_size,
                                                    class_mode='binary')
validation_generator = valid_datagen.flow_from_directory(valid_data_dir,
                                                         shuffle=False,
                                                         target_size=imsize,
                                                         batch_size=batch_size,
                                                         # class_mode="categorical" if there are more, than 2 classes
                                                         class_mode='binary'
                                                         )


Found 1594 images belonging to 2 classes.
Found 400 images belonging to 2 classes.


In [6]:
# Choose ResNet50 with pretrained imagenet weights as the base model
base_model = ResNet50(weights='imagenet', include_top=False)
# Get its output, add the average pooling layer and 1024 Dense layer
X = base_model.output
X = GlobalAveragePooling2D()(X)
X = Dense(1024, activation='relu')(X)
# Get the output
predictions = Dense(1, activation='sigmoid')(X)

/home/yurii/Work/study/tf/lib/python3.6/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [7]:
# Define a model
model = Model(inputs=base_model.input, outputs=predictions)
# Freeze the layers
for layer in base_model.layers:
    layer_trainable = False
# Compile the model, once it is ready to use
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [8]:
%%time

model.fit_generator(train_generator, train_generator.n // batch_size, epochs=epochs, workers=workers,
                   validation_data=validation_generator, validation_steps=validation_generator.n // batch_size)

Epoch 1/10
199/199 [==============================] - 50s 252ms/step - loss: 1.4047 - acc: 0.5308 - val_loss: 0.6937 - val_acc: 0.5000
Epoch 2/10
199/199 [==============================] - 41s 207ms/step - loss: 0.7132 - acc: 0.5358 - val_loss: 0.7598 - val_acc: 0.5425
Epoch 3/10
199/199 [==============================] - 41s 207ms/step - loss: 0.7014 - acc: 0.5320 - val_loss: 2.3110 - val_acc: 0.5225
Epoch 4/10
199/199 [==============================] - 41s 208ms/step - loss: 0.6936 - acc: 0.5402 - val_loss: 0.7393 - val_acc: 0.5300
Epoch 5/10
199/199 [==============================] - 41s 205ms/step - loss: 0.6988 - acc: 0.5138 - val_loss: 0.6730 - val_acc: 0.5875
Epoch 6/10
199/199 [==============================] - 42s 210ms/step - loss: 0.6886 - acc: 0.5540 - val_loss: 0.7067 - val_acc: 0.5225
Epoch 7/10
199/199 [==============================] - 41s 205ms/step - loss: 0.6908 - acc: 0.5421 - val_loss: 0.6911 - val_acc: 0.5250
Epoch 8/10
199/199 [==============================] - 4

In [9]:
# Fine-tune everything from a layer 140
split_at = 140
for layer in model.layers[:split_at]:
    layer.trainable = False
for layer in model.layers[split_at:]:
    layer.trainable = True
# Recompile the model
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [10]:
%%time
model.fit_generator(train_generator, train_generator.n // batch_size, epochs=1,
                    validation_data=validation_generator,
                    validation_steps=validation_generator.n // batch_size)

Epoch 1/1
199/199 [==============================] - 26s 130ms/step - loss: 0.6709 - acc: 0.5779 - val_loss: 0.7282 - val_acc: 0.5900
CPU times: user 38.2 s, sys: 3.29 s, total: 41.5 s
Wall time: 27.2 s


In [22]:
model.layersA